# The problem 
is about, finding most popular movies regarding Rating data. On the other hand, we need movie-name in rating. Instead of joining, we will broadcast movienames to all executers.

##### data structure for rating
user-id,movie-id,rating,timespamp

##### data structure for movie master data
movie-id|movie-name|movie-date|imdb-url|0|0|0|1|1|1|0|0|0|0|0|0|0|0|0|0|0|0|0

In [1]:
sc.version

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1555140556054_0001,pyspark,idle,Link,Link,✔


SparkSession available as 'spark'.
u'2.4.0'

In [9]:
### Movie Master Data
movie-id|movie-name|movie-date|imdb-url|0|0|0|1|1|1|0|0|0|0|0|0|0|0|0|0|0|0|0

VBox()

In [11]:
movies = sc.textFile("s3://spark-dataset-sandbox/datasets/ml-100k/u.item")

VBox()

In [15]:
movies.take(2)

VBox()

[u'1|Toy Story (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Toy%20Story%20(1995)|0|0|0|1|1|1|0|0|0|0|0|0|0|0|0|0|0|0|0', u'2|GoldenEye (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?GoldenEye%20(1995)|0|1|1|0|0|0|0|0|0|0|0|0|0|0|0|0|1|0|0']

In [10]:
### Rating Master Data
user-id,movie-id,rating,timespamp

VBox()

In [13]:
ratings = sc.textFile("s3://spark-dataset-sandbox/datasets/ml-100k/u.data")

VBox()

In [14]:
ratings.take(2)

VBox()

[u'196\t242\t3\t881250949', u'186\t302\t3\t891717742']